# Imports

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import tensorflow as tf
import tensorflow.keras as keras
import libpysal.weights as weights
import pysal.explore as esda
from utils.db_utils import engine

# Set random seed

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

# Load data

In [ ]:
raw_df = gpd.read_postgis("SELECT * FROM split_full_dataset", engine, "geometry")

In [ ]:
df = raw_df.copy()
df = df.drop(columns = ["ogc_fid"])

# Separate features

In [ ]:
features = raw_df.copy()
features["x_coord"] = features["geometry"].centroid.x
features["y_coord"] = features["geometry"].centroid.y
features = features.drop(columns=["geometry"])
labels = features.pop("very_good_health")

# Build model

In [ ]:
scores = []

In [ ]:
for i in range(10):

    # Build model
    normaliser = keras.layers.Normalization(axis=-1)
    normaliser.adapt(np.array(features))

    model = keras.Sequential([
        normaliser,
        keras.layers.Dense(372, activation="relu"),
        keras.layers.Dense(208, activation="relu"),
        keras.layers.Dense(173, activation="relu"),
        keras.layers.Dense(1)
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.015686), loss="mae"
    )

    # Build early stopper
    early_stopper = keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=20, restore_best_weights=True
    )

    # Fit model
    model.fit(
        features,
        labels,
        batch_size=30,
        epochs=200,
        validation_split=0.2,
        callbacks=[early_stopper],
        verbose=1,
    )

    # Calculate Moran's I of residuals and add to scores
    predictions = model.predict(features).flatten()
    residuals = labels - predictions
    features["residuals"] = residuals
    w = weights.KNN.from_dataframe(raw_df, k=8)
    moran = esda.esda.Moran(features["residuals"], w)
    scores.append(moran.I)

# Calculate Moran's I mean and standard deviation

In [ ]:
scores_df = pd.DataFrame(scores, columns = ["Moran's I"])
scores_df.loc["Mean"] = np.mean(scores)
scores_df.loc["Std dev"] = np.std(scores)
scores_df["Moran's I"] = scores_df["Moran's I"].round(6)

# Save output

In [ ]:
scores_df.to_csv("outputs/data_analyses/moran.csv")